<a href="https://colab.research.google.com/github/GoooIce/GoooIce/blob/main/interactive_infer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/FoundationVision/Infinity/

Cloning into 'Infinity'...
remote: Enumerating objects: 182, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 182 (delta 46), reused 154 (delta 28), pack-reused 4 (from 1)
Receiving objects: 100% (182/182), 9.54 MiB | 30.63 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [7]:
!wget https://huggingface.co/FoundationVision/Infinity/resolve/main/infinity_vae_d32reg.pth?download=true

--2025-01-03 14:26:47--  https://huggingface.co/FoundationVision/Infinity/resolve/main/infinity_vae_d32reg.pth?download=true
Resolving huggingface.co (huggingface.co)... 18.239.50.49, 18.239.50.103, 18.239.50.16, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.49|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/84/45/8445983988020af8980ce03b52cc249860ef6e4f4413e81ceac35347d4f7559e/7a37fa3ea1b2a1ebd23de61d91a5e68202825e5a67edaef4b7c55f5fd5b9cf26?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27infinity_vae_d32reg.pth%3B+filename%3D%22infinity_vae_d32reg.pth%22%3B&Expires=1736173608&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNjE3MzYwOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzg0LzQ1Lzg0NDU5ODM5ODgwMjBhZjg5ODBjZTAzYjUyY2MyNDk4NjBlZjZlNGY0NDEzZTgxY2VhYzM1MzQ3ZDRmNzU1OWUvN2EzN2ZhM2VhMWIyYTFlYmQyM2RlNjFkOTFhNWU2ODIwMjgyNWU1YTY3ZWRhZ

In [8]:
!wget https://huggingface.co/FoundationVision/Infinity/resolve/main/infinity_2b_reg.pth?download=true

--2025-01-03 14:28:36--  https://huggingface.co/FoundationVision/Infinity/resolve/main/infinity_2b_reg.pth?download=true
Resolving huggingface.co (huggingface.co)... 18.239.50.103, 18.239.50.49, 18.239.50.80, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.103|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/84/45/8445983988020af8980ce03b52cc249860ef6e4f4413e81ceac35347d4f7559e/d5958b37e7ecb914230cd2b8c28aae0ec108ce08b315720bc3dc52596cc5b3a7?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27infinity_2b_reg.pth%3B+filename%3D%22infinity_2b_reg.pth%22%3B&Expires=1736173716&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNjE3MzcxNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzg0LzQ1Lzg0NDU5ODM5ODgwMjBhZjg5ODBjZTAzYjUyY2MyNDk4NjBlZjZlNGY0NDEzZTgxY2VhYzM1MzQ3ZDRmNzU1OWUvZDU5NThiMzdlN2VjYjkxNDIzMGNkMmI4YzI4YWFlMGVjMTA4Y2UwOGIzMTU3MjBiYzNkYzUy

In [9]:
!cd Infinity

In [10]:
import random
import torch
torch.cuda.set_device(0)
import cv2
import numpy as np
from tools.run_infinity import *

model_path='../infinity_2b_reg.pth'
vae_path='../infinity_vae_d32_reg.pth'
text_encoder_ckpt = '../flan-t5-xl'
args=argparse.Namespace(
    pn='1M',
    model_path=model_path,
    cfg_insertion_layer=0,
    vae_type=32,
    vae_path=vae_path,
    add_lvl_embeding_only_first_block=1,
    use_bit_label=1,
    model_type='infinity_2b',
    rope2d_each_sa_layer=1,
    rope2d_normalized_by_hw=2,
    use_scale_schedule_embedding=0,
    sampling_per_bits=1,
    text_encoder_ckpt=text_encoder_ckpt,
    text_channels=2048,
    apply_spatial_patchify=0,
    h_div_w_template=1.000,
    use_flex_attn=0,
    cache_dir='/dev/shm',
    checkpoint_type='torch',
    seed=0,
    bf16=1,
    save_file='tmp.jpg'
)

ModuleNotFoundError: No module named 'tools'

In [ ]:
# load text encoder
text_tokenizer, text_encoder = load_tokenizer(t5_path=args.text_encoder_ckpt)
# load vae
vae = load_visual_tokenizer(args)
# load infinity
infinity = load_transformer(vae, args)

In [ ]:
prompt = """alien spaceship enterprise"""
cfg = 3
tau = 0.5
h_div_w = 1/1 # aspect ratio, height:width
seed = random.randint(0, 10000)
enable_positive_prompt=0

h_div_w_template_ = h_div_w_templates[np.argmin(np.abs(h_div_w_templates-h_div_w))]
scale_schedule = dynamic_resolution_h_w[h_div_w_template_][args.pn]['scales']
scale_schedule = [(1, h, w) for (_, h, w) in scale_schedule]
generated_image = gen_one_img(
    infinity,
    vae,
    text_tokenizer,
    text_encoder,
    prompt,
    g_seed=seed,
    gt_leak=0,
    gt_ls_Bl=None,
    cfg_list=cfg,
    tau_list=tau,
    scale_schedule=scale_schedule,
    cfg_insertion_layer=[args.cfg_insertion_layer],
    vae_type=args.vae_type,
    sampling_per_bits=args.sampling_per_bits,
    enable_positive_prompt=enable_positive_prompt,
)
args.save_file = 'ipynb_tmp.jpg'
os.makedirs(osp.dirname(osp.abspath(args.save_file)), exist_ok=True)
cv2.imwrite(args.save_file, generated_image.cpu().numpy())
print(f'Save to {osp.abspath(args.save_file)}')